In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
training_data.data[0].shape

torch.Size([28, 28])

In [4]:
training_data.data.shape

torch.Size([60000, 28, 28])

In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [51]:
# Get cpu, gpu or mps device for training.
# device = (
#     "cuda"
#     if torch.cuda.is_available()
#     else "mps"
#     if torch.backends.mps.is_available()
#     else "cpu"
# )
device = "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [52]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [53]:
import matplotlib.pyplot as plt
classes = [
        "T-shirt/top",
        "Trouser",
        "Pullover",
        "Dress",
        "Coat",
        "Sandal",
        "Shirt",
        "Sneaker",
        "Bag",
        "Ankle boot",
    ]

def display_sample(tensor_data, idx):
    plt.imshow(training_data[idx][0][0], cmap='gray')
    plt.title(classes[training_data[idx][1]])

In [54]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [57]:
def test(dataloader, model, loss_fn, verbose = False):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    num = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            if verbose & (num %  100 == 0):
                print(f'pred: {pred}')
                print(f'argmax: {pred.argmax(1)}')
                print(f'y: {y}')
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            num += 1
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct

In [58]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.416850  [   64/60000]
loss: 1.397583  [ 6464/60000]
loss: 1.219564  [12864/60000]
loss: 1.321375  [19264/60000]
loss: 1.207208  [25664/60000]
loss: 1.220680  [32064/60000]
loss: 1.232956  [38464/60000]
loss: 1.160596  [44864/60000]
loss: 1.210560  [51264/60000]
loss: 1.119666  [57664/60000]
Test Error: 
 Accuracy: 63.8%, Avg loss: 1.143482 

Epoch 2
-------------------------------
loss: 1.221830  [   64/60000]
loss: 1.217341  [ 6464/60000]
loss: 1.025467  [12864/60000]
loss: 1.154265  [19264/60000]
loss: 1.034073  [25664/60000]
loss: 1.066189  [32064/60000]
loss: 1.086176  [38464/60000]
loss: 1.027535  [44864/60000]
loss: 1.076706  [51264/60000]
loss: 0.999502  [57664/60000]
Test Error: 
 Accuracy: 65.0%, Avg loss: 1.017571 

Epoch 3
-------------------------------
loss: 1.085792  [   64/60000]
loss: 1.101648  [ 6464/60000]
loss: 0.894307  [12864/60000]
loss: 1.046492  [19264/60000]
loss: 0.927276  [25664/60000]
loss: 0.959111  [32064/600

In [59]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [60]:
test(test_dataloader, model, loss_fn)


Test Error: 
 Accuracy: 69.0%, Avg loss: 0.830726 



0.6895

In [61]:
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
